In [1]:
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import requests
import schedule
import pandas as pd
from datetime import datetime
from pathlib import Path

# === CONFIG ===
API_KEY = "583d2701f4394fa2a0a3d28addef4446"
BASE_URL = "https://api.opensea.io/api/v2/collections"
OUTPUT_FILE = Path("opensea_collection_stats.csv")

# file di output per analisi descrittive
LATEST_SNAPSHOT_FILE = Path("opensea_stats_latest.csv")
DESCRIBE_GLOBAL_FILE = Path("opensea_stats_describe_global.csv")
DESCRIBE_BY_SLUG_FILE = Path("opensea_stats_describe_by_slug.csv")
REPORT_HTML_FILE = Path("opensea_report.html")

# lista di collezioni da monitorare
COLLECTION_SLUGS = [
    "boredapeyachtclub",
    "doodles-official",
    "azuki",
    "hypurr-hyperevm"
]


def fetch_collection_stats(slug: str) -> dict:
    url = f"{BASE_URL}/{slug}/stats"
    headers = {
        "accept": "application/json",
    }
    if API_KEY:
        headers["x-api-key"] = API_KEY

    resp = requests.get(url, headers=headers, timeout=15)
    resp.raise_for_status()
    data = resp.json()

    total = data.get("total", {})

    # estraiamo le metriche interessanti + timestamp
    row = {
        "slug": slug,
        "timestamp_utc": datetime.utcnow().isoformat(),
        "volume": total.get("volume"),
        "sales": total.get("sales"),
        "num_owners": total.get("num_owners"),
        "market_cap": total.get("market_cap"),
        "floor_price": total.get("floor_price"),
        "floor_price_symbol": total.get("floor_price_symbol"),
        "average_price": total.get("average_price"),
    }

    return row


def generate_html_report(
    df: pd.DataFrame,
    latest_per_slug: pd.DataFrame,
    describe_global: pd.DataFrame,
    describe_by_slug: pd.DataFrame,
    output_path: Path = REPORT_HTML_FILE,
) -> None:
    """
    Genera un report HTML con:
    - info generali sul dataset
    - ultima snapshot per slug
    - statistiche globali
    - statistiche per slug
    """
    now_str = datetime.utcnow().isoformat()

    # info generali
    n_rows = len(df)
    n_slugs = df["slug"].nunique()
    slugs_list = ", ".join(sorted(df["slug"].unique()))

    ts_min = df["timestamp_utc"].min()
    ts_max = df["timestamp_utc"].max()

    latest_html = latest_per_slug.to_html(
        index=False,
        border=0,
        classes="table table-striped table-sm"
    )
    global_html = describe_global.to_html(
        border=0,
        classes="table table-striped table-sm"
    )
    by_slug_html = describe_by_slug.to_html(
        border=0,
        classes="table table-striped table-sm"
    )

    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>OpenSea Collections Report</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #0b0c10;
            color: #f8f8f2;
        }}
        h1, h2, h3 {{
            color: #66fcf1;
        }}
        .meta {{
            font-size: 0.9rem;
            margin-bottom: 20px;
        }}
        .section {{
            margin-bottom: 40px;
        }}
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-top: 10px;
            background-color: #1f2833;
        }}
        th, td {{
            padding: 8px 10px;
            border: 1px solid #45a29e;
            font-size: 0.85rem;
        }}
        th {{
            background-color: #45a29e;
            color: #0b0c10;
        }}
        .badge {{
            display: inline-block;
            padding: 4px 8px;
            margin: 2px;
            border-radius: 999px;
            background-color: #45a29e;
            color: #0b0c10;
            font-size: 0.8rem;
        }}
        .pill-container {{
            margin-top: 5px;
            margin-bottom: 15px;
        }}
        .small {{
            font-size: 0.8rem;
            opacity: 0.85;
        }}
    </style>
</head>
<body>
    <h1>OpenSea Collections Daily Report</h1>
    <div class="meta">
        <div><strong>Generato (UTC):</strong> {now_str}</div>
        <div><strong>Righe totali:</strong> {n_rows}</div>
        <div><strong>Collezioni monitorate:</strong> {n_slugs}</div>
        <div><strong>Range temporale (UTC):</strong> {ts_min} → {ts_max}</div>
        <div class="pill-container">
            <span class="small"><strong>Slug:</strong></span><br/>
            {"".join(f'<span class="badge">{s}</span>' for s in slugs_list.split(", "))}
        </div>
    </div>

    <div class="section">
        <h2>Ultimo snapshot per collezione</h2>
        <p class="small">
            Ogni riga rappresenta l'osservazione più recente per quella collezione,
            basata sul campo <code>timestamp_utc</code>.
        </p>
        {latest_html}
    </div>

    <div class="section">
        <h2>Statistiche descrittive globali</h2>
        <p class="small">
            Statistiche su tutte le osservazioni del dataset per le principali
            metriche numeriche (volume, sales, num_owners, market_cap, floor_price, average_price).
        </p>
        {global_html}
    </div>

    <div class="section">
        <h2>Statistiche descrittive per collezione</h2>
        <p class="small">
            Statistiche raggruppate per <code>slug</code>. Ogni blocco di righe corrisponde
            a una collezione.
        </p>
        {by_slug_html}
    </div>
</body>
</html>
"""

    output_path.write_text(html, encoding="utf-8")
    print(f"[REPORT] Report HTML generato in {output_path}")


def run_descriptive_analytics(csv_path: Path) -> None:
    """
    Legge tutto lo storico dal CSV e calcola alcune analisi descrittive statiche.
    Salva i risultati in altri CSV di comodo + un report HTML.
    """
    if not csv_path.exists():
        print(f"[ANALISI] Nessun file {csv_path}, salto le analisi.")
        return

    df = pd.read_csv(csv_path)

    if df.empty:
        print("[ANALISI] DataFrame vuoto, nessuna analisi da fare.")
        return

    # parsing del timestamp
    df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], errors="coerce")

    numeric_cols = [
        "volume",
        "sales",
        "num_owners",
        "market_cap",
        "floor_price",
        "average_price",
    ]

    # 1) Ultimo snapshot per slug (l’osservazione più recente)
    latest_per_slug = (
        df.sort_values("timestamp_utc")
          .groupby("slug", as_index=False)
          .tail(1)
    )
    latest_per_slug.to_csv(LATEST_SNAPSHOT_FILE, index=False)
    print(f"[ANALISI] Salvato ultimo snapshot per slug in {LATEST_SNAPSHOT_FILE}")

    # 2) Statistiche descrittive globali sulle colonne numeriche
    describe_global = df[numeric_cols].describe()
    describe_global.to_csv(DESCRIBE_GLOBAL_FILE)
    print(f"[ANALISI] Salvate statistiche globali in {DESCRIBE_GLOBAL_FILE}")

    # 3) Statistiche descrittive per slug
    describe_by_slug = df.groupby("slug")[numeric_cols].describe()
    describe_by_slug.to_csv(DESCRIBE_BY_SLUG_FILE)
    print(f"[ANALISI] Salvate statistiche per slug in {DESCRIBE_BY_SLUG_FILE}")

    # 4) Report HTML
    generate_html_report(df, latest_per_slug, describe_global, describe_by_slug)

    # Optional: qualche preview in console
    print("\n[ANALISI] Statistiche globali (preview):")
    print(describe_global)

    print("\n[ANALISI] Ultimo snapshot per slug (preview):")
    print(latest_per_slug[["slug", "timestamp_utc", "floor_price", "market_cap"]])


def run_etl():
    rows = []

    for slug in COLLECTION_SLUGS:
        try:
            stats = fetch_collection_stats(slug)
            rows.append(stats)
        except Exception as e:
            print(f"[ERRORE] slug={slug} -> {e}")

    if not rows:
        print("Nessun dato estratto, ETL terminata.")
        return

    df = pd.DataFrame(rows)

    # se il file esiste già append, altrimenti crea con header
    file_exists = OUTPUT_FILE.exists()
    df.to_csv(OUTPUT_FILE, mode="a", header=not file_exists, index=False)
    print(f"Salvate {len(df)} righe in {OUTPUT_FILE}")

    # dopo l’update dello storico, lancia le analisi descrittive + report HTML
    run_descriptive_analytics(OUTPUT_FILE)


def job():
    """
    Job schedulato: esegue ETL + analisi descrittiva + HTML report.
    """
    print(f"\n=== JOB START {datetime.utcnow().isoformat()} UTC ===")
    run_etl()
    print(f"=== JOB END {datetime.utcnow().isoformat()} UTC ===\n")


if __name__ == "__main__":
    # imposta l'orario giornaliero (formato HH:MM, 24h)
    # cambia "09:00" con l’orario che preferisci
    schedule.every().day.at("11:00").do(job)

    print("Scheduler avviato. In attesa del prossimo run giornaliero...")

    try:
        while True:
            schedule.run_pending()
            time.sleep(60)  # controlla i job ogni 60 secondi
    except KeyboardInterrupt:
        print("\nScheduler interrotto manualmente.")


Scheduler avviato. In attesa del prossimo run giornaliero...

=== JOB START 2025-11-21T10:00:54.553719 UTC ===
Salvate 4 righe in opensea_collection_stats.csv
[ANALISI] Salvato ultimo snapshot per slug in opensea_stats_latest.csv
[ANALISI] Salvate statistiche globali in opensea_stats_describe_global.csv
[ANALISI] Salvate statistiche per slug in opensea_stats_describe_by_slug.csv
[REPORT] Report HTML generato in opensea_report.html

[ANALISI] Statistiche globali (preview):
             volume         sales   num_owners    market_cap  floor_price  \
count  1.700000e+01     17.000000    17.000000     17.000000    17.000000   
mean   8.127032e+05  66856.588235  4630.470588   7955.614343    79.228912   
std    5.569788e+05  27626.342124   604.655803   6040.770363   216.725559   
min    6.371086e+04   4148.000000  3794.000000    505.605194     0.580000   
25%    3.518900e+05  54508.000000  4332.000000   1534.164362     0.580000   
50%    8.275980e+05  82720.000000  4399.000000  10516.933677 